In [1]:
import pandas as pd
import numpy as np
import os
import dpsim

Could not find dpsim C++ module.


In [2]:
import requests
url = 'https://git.rwth-aachen.de/acs/public/simulation/reference-results/-/raw/master/Matpower/case9results.mat'
with open('case9results.mat', 'wb') as out_file:
    content = requests.get(url, stream=True).content
    out_file.write(content)

In [3]:
mpc_reader = dpsim.matpower.Reader('case9results.mat', 'case9results')

In [4]:
system = mpc_reader.load_mpc()

[12:01:12.664912 line1_1-4 warning] Zero value for Capacitance, setting default value of C=1e-12 [F]
[12:01:12.664949 line1_1-4 warning] Zero value for Conductance, setting default value of G=1e-06 [S]
[12:01:12.666493 line2_4-5 warning] Zero value for Conductance, setting default value of G=1e-06 [S]
[12:01:12.668002 line3_5-6 warning] Zero value for Conductance, setting default value of G=1e-06 [S]
[12:01:12.669511 line4_3-6 warning] Zero value for Capacitance, setting default value of C=1e-12 [F]
[12:01:12.669515 line4_3-6 warning] Zero value for Conductance, setting default value of G=1e-06 [S]
[12:01:12.670975 line5_6-7 warning] Zero value for Conductance, setting default value of G=1e-06 [S]
[12:01:12.672329 line6_7-8 warning] Zero value for Conductance, setting default value of G=1e-06 [S]
[12:01:12.673695 line7_8-2 warning] Zero value for Capacitance, setting default value of C=1e-12 [F]
[12:01:12.673700 line7_8-2 warning] Zero value for Conductance, setting default value of G=

In [7]:
sim_name = 'case9'
dpsim.Logger.set_log_dir('logs/' + sim_name)
logger = dpsim.Logger(sim_name)

for node in system.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
    logger.log_attribute(node.name()+'.S', 's', node)
    
# Parametrize and run simulation
sim = dpsim.Simulation(sim_name, dpsim.LogLevel.info)
sim.set_system(system)
sim.set_time_step(1)
sim.set_final_time(1)
sim.set_domain(dpsim.Domain.SP)
sim.set_solver(dpsim.Solver.NRP)
sim.do_init_from_nodes_and_terminals(False)
sim.add_logger(logger)
sim.run()

[12:02:08.744676 case9 info] Initialize simulation: case9
[12:02:08.745294 case9_PF_Solver warning] Unable to get base voltage at 1
[12:02:08.745322 case9_PF_Solver warning] Unable to get base voltage at 4
[12:02:08.745328 case9_PF_Solver warning] Unable to get base voltage at 5
[12:02:08.745334 case9_PF_Solver warning] Unable to get base voltage at 6
[12:02:08.745340 case9_PF_Solver warning] Unable to get base voltage at 7
[12:02:08.745346 case9_PF_Solver warning] Unable to get base voltage at 8
[12:02:08.745352 case9_PF_Solver warning] Unable to get base voltage at 9
[12:02:08.745774 case9 info] Scheduling tasks.
[12:02:08.746010 case9 info] Scheduling done.
[12:02:08.746012 case9 info] Opening interfaces.
[12:02:08.746015 case9 info] Start synchronization with remotes on 0 interfaces
[12:02:08.746016 case9 info] Synchronized simulation start with remotes
[12:02:08.746018 case9 info] Start simulation: case9
[12:02:08.746023 case9 info] Time step: 1.000000e+00
[12:02:08.746024 case9 i

In [8]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts

log_file = 'logs/' + sim_name +'/' + sim_name + '.csv'
print(log_file)
ts_dpsimpy = rt.read_timeseries_dpsim(log_file)

logs/case9/case9.csv
column number: 18
results length: 1
real column names: []
complex column names: ['1.S', '1.V', '2.S', '2.V', '3.S', '3.V', '4.S', '4.V', '5.S', '5.V', '6.S', '6.V', '7.S', '7.V', '8.S', '8.V', '9.S', '9.V']


In [9]:
results = pd.DataFrame(columns=['Bus', 'V_mag(pu)', 'V_angle(deg)', 'P(MW)', 'Q (MVAr)'])

for i in range(len(system.nodes)):
    node = system.nodes[i].name()
    node_baseV = mpc_reader.mpc_bus_data.loc[mpc_reader.mpc_bus_data['bus_i'] == int(node), 'baseKV'].iloc[0] * 1e3
    w_mw = 1e-6
    results.loc[i] = ([node] + [np.absolute(ts_dpsimpy[node + '.V'].values[-1]) / node_baseV]
        + [np.degrees(np.angle(ts_dpsimpy[node + '.V'].values[-1]))] 
        + [w_mw * np.real(ts_dpsimpy[node + '.S'].values[-1])] 
        + [w_mw * np.imag(ts_dpsimpy[node + '.S'].values[-1])])

In [10]:
results

,Bus,V_mag(pu),V_angle(deg),P(MW),Q (MVAr)
0,1,1.040000,0.000000,72.12808,27.032265
1,2,1.025000,9.242530,163.00000,6.648797
2,3,1.025000,4.627326,85.00000,-10.855839
3,4,1.025806,-2.230943,0.00000,0.000000
4,5,1.012668,-3.711116,-90.00000,-30.000000
5,6,1.032350,1.930165,0.00000,0.000000
6,7,1.015878,0.689738,-100.00000,-35.000000
7,8,1.025771,3.683184,0.00000,0.000000
8,9,0.995654,-4.012092,-125.00000,-50.000000


In [11]:
import scipy.io
input_mat = 'case9results.mat'
data = scipy.io.loadmat(input_mat)
 
bus_data = pd.DataFrame(data['case9results'][0][0]['bus'])
bus_data.columns = ["bus_i", "type", "Pd", "Qd", "Gs", "Bs", "area", "Vm", "Va", "baseKV", "zone", "Vmax", "Vmin"]

In [17]:
bus_data

,bus_i,type,Pd,Qd,Gs,Bs,area,Vm,Va,baseKV,zone,Vmax,Vmin
0,1.0,3.0,0.0,0.0,0.0,0.0,1.0,1.040000,0.000000,230.0,1.0,1.1,0.9
1,2.0,2.0,0.0,0.0,0.0,0.0,1.0,1.025000,9.280005,230.0,1.0,1.1,0.9
2,3.0,2.0,0.0,0.0,0.0,0.0,1.0,1.025000,4.664751,230.0,1.0,1.1,0.9
3,4.0,1.0,0.0,0.0,0.0,0.0,1.0,1.025788,-2.216788,230.0,1.0,1.1,0.9
4,5.0,1.0,90.0,30.0,0.0,0.0,1.0,1.012654,-3.687396,230.0,1.0,1.1,0.9
5,6.0,1.0,0.0,0.0,0.0,0.0,1.0,1.032353,1.966716,230.0,1.0,1.1,0.9
6,7.0,1.0,100.0,35.0,0.0,0.0,1.0,1.015883,0.727536,230.0,1.0,1.1,0.9
7,8.0,1.0,0.0,0.0,0.0,0.0,1.0,1.025769,3.719701,230.0,1.0,1.1,0.9
8,9.0,1.0,125.0,50.0,0.0,0.0,1.0,0.995631,-3.988805,230.0,1.0,1.1,0.9


In [18]:
assert all(bus_data['Vm'] - results['V_mag(pu)'] < 0.001)

In [19]:
assert all(bus_data['Va'] - results['V_angle(deg)'] < 0.05)